In [1]:
import networkx as nx
import numpy as np
from grandiso import find_motifs

In [2]:
def nb_samples_required(a, delta, epsilon):
    return int(np.ceil(2 * (np.log(2) * a + np.log(1 / delta)) / (epsilon**2)))

In [54]:
# Create all graphlets of size 3, 4, 5
def build_graphlets(k):
    if k == 3:
        graphlets = [
            np.zeros((3, 3)),
            np.array([[0, 1, 0], [1, 0, 0], [0, 0, 0]]),
            np.array([[0, 1, 0], [1, 0, 1], [0, 1, 0]]),
            np.array([[0, 1, 0], [0, 0, 1], [1, 0, 0]])
        ]
    if k == 4:
        graphlets = [
            np.zeros((4,4)),
            np.array([[0, 1, 0, 0],[1, 0, 0, 0],[0, 0, 0, 0],[0, 0, 0, 0]]),
            np.array([[0, 1, 0, 0],[1, 0, 0, 0],[0, 0, 0, 1],[0, 0, 1, 0]]),
            np.array([[0, 1, 1, 0],[1, 0, 0, 0],[1, 0, 0, 0],[0, 0, 0, 0]]),
            np.array([[0, 1, 1, 0],[1, 0, 1, 0],[1, 1, 0, 0],[0, 0, 0, 0]]),
            np.array([[0, 1, 1, 0],[1, 0, 0, 1],[1, 0, 0, 0],[0, 1, 0, 0]]),
            np.array([[0, 1, 1, 0],[1, 0, 0, 1],[1, 0, 1, 0],[0, 1, 1, 0]]),
            np.array([[0, 1, 1, 1],[1, 0, 0, 0],[1, 0, 0, 0],[1, 0, 0, 0]]),
            np.array([[0, 1, 1, 1],[1, 0, 1, 0],[1, 1, 0, 0],[1, 0, 0, 0]]),
            np.array([[0, 1, 1, 1],[1, 0, 1, 1],[1, 1, 0, 0],[1, 1, 0, 0]]),
            np.array([[0, 1, 1, 1],[1, 0, 1, 1],[1, 1, 0, 1],[1, 1, 1, 0]])
        ]
        
        
        np.array([[0, 0, 0, 0, 0],[0, 0, 0, 0, 0],[0, 0, 0, 0, 0],[0, 0, 0, 0, 0],[0, 0, 0, 0, 0]]),
        
    if k == 5:
        graphlets = [
            np.zeros((5,5)),
            np.array([[0, 1, 0, 0, 0],[1, 0, 0, 0, 0],[0, 0, 0, 0, 0],[0, 0, 0, 0, 0],[0, 0, 0, 0, 0]]), #2
            np.array([[0, 1, 0, 0, 0],[1, 0, 1, 0, 0],[0, 1, 0, 0, 0],[0, 0, 0, 0, 0],[0, 0, 0, 0, 0]]), #3
            np.array([[0, 1, 0, 0, 0],[1, 0, 0, 0, 0],[0, 0, 0, 1, 0],[0, 0, 1, 0, 0],[0, 0, 0, 0, 0]]), #4
            np.array([[0, 1, 1, 1, 0],[1, 0, 0, 0, 0],[1, 0, 0, 0, 0],[1, 0, 0, 0, 0],[0, 0, 0, 0, 0]]), #5
            np.array([[0, 1, 1, 0, 0],[1, 0, 1, 0, 0],[1, 1, 0, 0, 0],[0, 0, 0, 0, 0],[0, 0, 0, 0, 0]]), #6
            np.array([[0, 1, 0, 0, 0],[1, 0, 1, 0, 0],[0, 1, 0, 1, 0],[0, 0, 1, 0, 0],[0, 0, 0, 0, 0]]), #7
            np.array([[0, 1, 0, 0, 0],[1, 0, 1, 0, 0],[0, 1, 0, 0, 0],[0, 0, 0, 0, 1],[0, 0, 0, 1, 0]]), #8
            np.array([[0, 1, 1, 1, 1],[1, 0, 0, 0, 0],[1, 0, 0, 0, 0],[1, 0, 0, 0, 0],[1, 0, 0, 0, 0]]), #9
            np.array([[0, 1, 1, 1, 0],[1, 0, 1, 0, 0],[1, 1, 0, 0, 0],[1, 0, 0, 0, 0],[0, 0, 0, 0, 0]]), #10
            np.array([[0, 1, 0, 0, 0],[1, 0, 1, 0, 0],[0, 1, 0, 1, 1],[0, 0, 1, 0, 0],[0, 0, 1, 0, 0]]), #11
            np.array([[0, 1, 1, 0, 0],[1, 0, 1, 0, 0],[1, 1, 0, 0, 0],[0, 0, 0, 0, 1],[0, 0, 0, 1, 0]]), #12 
            np.array([[0, 1, 0, 1, 0],[1, 0, 1, 0, 0],[0, 1, 0, 1, 0],[1, 0, 1, 0, 0],[0, 0, 0, 0, 0]]), #13
            np.array([[0, 1, 0, 0, 0],[1, 0, 1, 0, 0],[0, 1, 0, 1, 0],[0, 0, 1, 0, 1],[0, 0, 0, 1, 0]]), #14
            np.array([[0, 1, 1, 1, 1],[1, 0, 1, 0, 0],[1, 1, 0, 0, 0],[1, 0, 0, 0, 0],[1, 0, 0, 0, 0]]), #15
            np.array([[0, 1, 1, 0, 1],[1, 0, 1, 0, 0],[1, 1, 0, 1, 0],[1, 0, 1, 0, 0],[0, 0, 0, 0, 0]]), #16
            np.array([[0, 1, 1, 1, 0],[1, 0, 1, 0, 1],[1, 1, 0, 0, 0],[1, 0, 0, 0, 0],[0, 1, 0, 0, 0]]), #17
            np.array([[0, 1, 1, 1, 0],[1, 0, 1, 0, 0],[1, 1, 0, 0, 0],[1, 0, 0, 0, 1],[0, 0, 0, 1, 0]]), #18
            np.array([[0, 1, 0, 1, 1],[1, 0, 1, 0, 0],[0, 1, 0, 1, 0],[1, 0, 1, 0, 0],[1, 0, 0, 0, 0]]), #19
            np.array([[0, 1, 0, 0, 1],[1, 0, 1, 0, 0],[0, 1, 0, 1, 0],[0, 0, 1, 0, 1],[1, 0, 0, 1, 0]]), #20
            np.array([[0, 1, 1, 1, 1],[1, 0, 1, 0, 0],[1, 1, 0, 1, 0],[1, 0, 1, 0, 0],[1, 0, 0, 0, 0]]), #21
            np.array([[0, 1, 1, 1, 1],[1, 0, 1, 0, 0],[1, 1, 0, 0, 0],[1, 0, 0, 0, 1],[1, 0, 0, 1, 0]]), #22
            np.array([[0, 1, 3, 1, 0],[1, 0, 1, 1, 0],[1, 1, 0, 1, 0],[1, 1, 1, 0, 0],[0, 0, 0, 0, 0]]), #23
            np.array([[0, 1, 0, 1, 1],[1, 0, 1, 1, 0],[0, 1, 0, 1, 0],[1, 1, 1, 0, 0],[1, 0, 0, 0, 0]]), #24
            np.array([[0, 1, 0, 1, 1],[1, 0, 1, 0, 1],[0, 1, 0, 1, 0],[1, 0, 1, 0, 0],[1, 1, 0, 0, 0]]), #25
            np.array([[0, 1, 0, 1, 1],[1, 0, 1, 0, 0],[0, 1, 0, 1, 1],[1, 0, 1, 0, 0],[1, 0, 1, 0, 0]]), #26
            np.array([[0, 1, 1, 1, 1],[1, 0, 1, 0, 0],[1, 1, 0, 1, 1],[1, 0, 1, 0, 0],[1, 0, 1, 0, 0]]), #27
            np.array([[0, 1, 3, 1, 0],[1, 0, 1, 1, 1],[1, 1, 0, 1, 0],[1, 1, 1, 0, 0],[0, 1, 0, 0, 0]]), #28
            np.array([[0, 1, 1, 1, 1],[1, 0, 1, 1, 0],[1, 1, 0, 0, 0],[1, 1, 0, 0, 1],[1, 0, 0, 1, 0]]), #29
            np.array([[0, 1, 1, 0, 1],[1, 0, 1, 1, 0],[1, 1, 0, 1, 0],[1, 0, 1, 0, 1],[1, 0, 0, 1, 0]]), #30
            np.array([[0, 1, 1, 1, 1],[1, 0, 1, 0, 0],[1, 1, 0, 1, 1],[1, 0, 1, 0, 1],[1, 0, 1, 1, 0]]), #31
            np.array([[0, 1, 0, 1, 1],[1, 0, 1, 0, 1],[0, 1, 0, 1, 1],[1, 0, 1, 0, 1],[1, 1, 1, 1, 0]]), #32
            np.array([[0, 1, 1, 1, 1],[1, 0, 1, 1, 0],[1, 1, 0, 1, 1],[1, 1, 1, 0, 1],[1, 0, 1, 1, 0]]), #33
            np.array([[0, 1, 1, 1, 1],[1, 0, 1, 1, 1],[1, 1, 0, 1, 1],[1, 1, 1, 0, 1],[1, 1, 1, 1, 0]]), #33
        ]
    
    return [nx.from_numpy_matrix(m) for m in graphlets]

In [4]:
def sample_subgraphs(G, k):
    nodes = G.nodes()

    return G.subgraph(np.random.choice(nodes, k, replace=False).tolist())

In [5]:
def find_iso_graphlet(subgraph, graphlets, spectrum):
    for i, graphlet in enumerate(graphlets):
        if nx.is_isomorphic(subgraph, graphlet):
            spectrum[i] += 1

In [49]:
def estimate_spectrum(G, k, delta, epsilon):
    graphlets = build_graphlets(k)
    m = nb_samples_required(len(graphlets), delta, epsilon)
    spectrum = np.zeros(len(graphlets))

    for _ in range(m):
        subgraph = sample_subgraphs(G, k)
        find_iso_graphlet(subgraph, graphlets, spectrum)

    #return spectrum / np.linalg.norm(spectrum) # to sum to 1 for kernel(G, G)
    return spectrum / m

In [7]:
def kernel(G1, G2, k, delta, epsilon):
    spectrum1 = estimate_spectrum(G1, k, delta, epsilon)
    spectrum2 = estimate_spectrum(G2, k, delta, epsilon)

    return spectrum1.T @ spectrum2

In [61]:
G = nx.fast_gnp_random_graph(100, 0.1)

spectrum = estimate_spectrum(G, 5, 0.05, 0.05)

In [62]:
sum(spectrum)

0.9999529433908992

In [10]:
kernel(G, G, 3, 0.05, 0.05)

0.5902935573269605

In [36]:
graphlet = nx.from_numpy_matrix(np.array([[0, 1, 0], [0, 0, 1], [1, 0, 0]]))
motif1 = nx.Graph()
motif1.add_node("A")
motif1.add_node("C")
motif1.add_edge("A","C")
motif2 = nx.Graph()
motif2.add_edge("A", "B")
motif2.add_edge("B", "C")
motif2.add_edge("C", "A")

find_motifs(motif1,G,count_only=True,isomorphisms_only=True),find_motifs(motif2,G,count_only=True,isomorphisms_only=True)



(956, 924)